In [1]:

library(caret, quiet=TRUE);
library(base64enc)
library(httr)



Attaching package: ‘httr’

The following object is masked from ‘package:caret’:

    progress



# Build a Model

In [2]:

## binary classification in iris dataset:
    
num_class <- 3
lb <- as.numeric(iris$Species) - 1

iris$Species2 = as.factor(sapply(iris$Species , function(x) if( x == 'setosa') 'yes' else 'no'))
dataset = as.matrix(iris[, 1:4])
    
create_model  =  function() {

    set.seed(1960)
    formula <- as.formula(Species2 ~ Sepal.Length + Sepal.Width + Petal.Length + Petal.Width)
    model <- train(formula, iris, method = "glm", maxit = 100, trace=FALSE)
    return(model)
}

In [3]:
head(iris)

Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,Species2
5.1,3.5,1.4,0.2,setosa,yes
4.9,3.0,1.4,0.2,setosa,yes
4.7,3.2,1.3,0.2,setosa,yes
4.6,3.1,1.5,0.2,setosa,yes
5.0,3.6,1.4,0.2,setosa,yes
5.4,3.9,1.7,0.4,setosa,yes


In [4]:
# dataset
model = create_model()
# cat(model$feature_names)
# plot(model)


Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning message:
“glm.fit: fitted probabilities numerically 0 or 1 occurred”Warning mess

In [5]:
pred <- predict(model, dataset , type="prob")
pred_labels <- predict(model, dataset , type="raw")
error_rate = sum(pred_labels != iris$Species2)/length(pred_labels)
error_rate

[1] 0

# SQL Code Generation

In [6]:

test_ws_sql_gen = function(mod) {
    WS_URL = "https://sklearn2sql.herokuapp.com/model"
    WS_URL = "http://localhost:1888/model"
    model_serialized <- serialize(mod, NULL)
    b64_data = base64encode(model_serialized)
    data = list(Name = "xgboost_test_model", SerializedModel = b64_data , SQLDialect = "postgresql" , Mode="xgboost+r")
    r = POST(WS_URL, body = data, encode = "json", verbose())
    # print(r)
    content = content(r)
    # print(content)
    lSQL = content$model$SQLGenrationResult[[1]]$SQL # content["model"]["SQLGenrationResult"][0]["SQL"]
    return(lSQL);
}

In [7]:
# labels(model$finalModel)

In [8]:
lModelSQL = test_ws_sql_gen(model)
cat(lModelSQL)
















































































WITH linear_input AS 
(SELECT "ADS"."KEY" AS "KEY", CAST("ADS"."Feature_0" AS FLOAT) AS "Feature_0", CAST("ADS"."Feature_1" AS FLOAT) AS "Feature_1", CAST("ADS"."Feature_2" AS FLOAT) AS "Feature_2", CAST("ADS"."Feature_3" AS FLOAT) AS "Feature_3" 
FROM "INPUT_DATA" AS "ADS"), 
linear_model_cte AS 
(SELECT linear_input."KEY" AS "KEY", 12.889612918026293 * linear_input."Feature_0" + 8.315749215379096 * linear_input."Feature_1" + -21.94182207054124 * linear_input."Feature_2" + -23.62814731180407 * linear_input."Feature_3" + -18.110988433258868 AS lincomb 
FROM linear_input), 
dot_prod_logistic AS 
(SELECT linear_model_cte."KEY" AS "KEY", linear_model_cte.lincomb AS dot_prod, 1.0 / (1.0 + exp(least(greatest(-100.0, -linear_model_cte.lincomb), 100.0))) AS logistic 
FROM linear_model_cte)
 SELECT dot_prod_logistic."KEY" AS "KEY", CAST(NULL AS FLOAT) AS "Score_no", CAST(NULL AS FLOAT) AS "Score_yes", 1.0 - dot_prod

# Execute the SQL Code

In [9]:
library(RODBC)
conn = odbcConnect("pgsql", uid="db", pwd="db", case="nochange")
odbcSetAutoCommit(conn , autoCommit = TRUE)

[1] 0

In [10]:
df_sql = as.data.frame(dataset)
names(df_sql) = sprintf("Feature_%d",0:(ncol(df_sql)-1))
df_sql$KEY = seq.int(nrow(dataset))

sqlDrop(conn , "INPUT_DATA" , errors = FALSE)
sqlSave(conn, df_sql, tablename = "INPUT_DATA", verbose = FALSE)

head(df_sql)

Feature_0,Feature_1,Feature_2,Feature_3,KEY
5.1,3.5,1.4,0.2,1
4.9,3.0,1.4,0.2,2
4.7,3.2,1.3,0.2,3
4.6,3.1,1.5,0.2,4
5.0,3.6,1.4,0.2,5
5.4,3.9,1.7,0.4,6


In [11]:
# colnames(df_sql)
# odbcGetInfo(conn)
# sqlTables(conn)

In [12]:
df_sql_out = sqlQuery(conn, lModelSQL)
head(df_sql_out)

KEY,Score_no,Score_yes,Proba_no,Proba_yes,LogProba_no,LogProba_yes,Decision,DecisionProba
1,NA,NA,0.000000e+00,1,-1.797693e+308,0.000000e+00,yes,1
2,NA,NA,8.881784e-16,1,-3.465736e+01,-8.881784e-16,yes,1
3,NA,NA,2.220446e-16,1,-3.604365e+01,-2.220446e-16,yes,1
4,NA,NA,1.842970e-13,1,-2.932223e+01,-1.842970e-13,yes,1
5,NA,NA,0.000000e+00,1,-1.797693e+308,0.000000e+00,yes,1
6,NA,NA,0.000000e+00,1,-1.797693e+308,0.000000e+00,yes,1


In [13]:
# colnames(df1)

# R XGBoost Output

In [14]:
pred  =  predict(model, as.matrix(dataset), type = "prob")
# pred <- matrix(pred , ncol=3, byrow=TRUE)
# p0 = 1 - pred
df_r_out = data.frame(pred)
names(df_r_out) = sprintf("Proba_%s",model$levels)

df_r_out$KEY = seq.int(nrow(dataset))
df_r_out$Score_no  =  NA
df_r_out$Score_yes  =  NA
df_r_out$LogProba_no  =  log(df_r_out$Proba_no)
df_r_out$LogProba_yes  =  log(df_r_out$Proba_yes)
df_r_out$Decision =   predict(model, as.matrix(dataset), type = "raw")
df_r_out$DecisionProba =  apply(pred, 1, function(x) max(x))
head(df_r_out)


Proba_no,Proba_yes,KEY,Score_no,Score_yes,LogProba_no,LogProba_yes,Decision,DecisionProba
2.220446e-16,1,1,NA,NA,-36.04365,-2.220446e-16,yes,1
2.220446e-16,1,2,NA,NA,-36.04365,-2.220446e-16,yes,1
2.220446e-16,1,3,NA,NA,-36.04365,-2.220446e-16,yes,1
1.844080e-13,1,4,NA,NA,-29.32163,-1.844080e-13,yes,1
2.220446e-16,1,5,NA,NA,-36.04365,-2.220446e-16,yes,1
2.220446e-16,1,6,NA,NA,-36.04365,-2.220446e-16,yes,1


In [15]:
# df_r_out

# Compare R and SQL output

In [16]:
df_merge = merge(x = df_r_out, y = df_sql_out, by = "KEY", all = TRUE, , suffixes = c("_1","_2"))
head(df_merge)

KEY,Proba_no_1,Proba_yes_1,Score_no_1,Score_yes_1,LogProba_no_1,LogProba_yes_1,Decision_1,DecisionProba_1,Score_no_2,Score_yes_2,Proba_no_2,Proba_yes_2,LogProba_no_2,LogProba_yes_2,Decision_2,DecisionProba_2
1,2.220446e-16,1,NA,NA,-36.04365,-2.220446e-16,yes,1,NA,NA,0.000000e+00,1,-1.797693e+308,0.000000e+00,yes,1
2,2.220446e-16,1,NA,NA,-36.04365,-2.220446e-16,yes,1,NA,NA,8.881784e-16,1,-3.465736e+01,-8.881784e-16,yes,1
3,2.220446e-16,1,NA,NA,-36.04365,-2.220446e-16,yes,1,NA,NA,2.220446e-16,1,-3.604365e+01,-2.220446e-16,yes,1
4,1.844080e-13,1,NA,NA,-29.32163,-1.844080e-13,yes,1,NA,NA,1.842970e-13,1,-2.932223e+01,-1.842970e-13,yes,1
5,2.220446e-16,1,NA,NA,-36.04365,-2.220446e-16,yes,1,NA,NA,0.000000e+00,1,-1.797693e+308,0.000000e+00,yes,1
6,2.220446e-16,1,NA,NA,-36.04365,-2.220446e-16,yes,1,NA,NA,0.000000e+00,1,-1.797693e+308,0.000000e+00,yes,1


In [17]:
diffs_df = df_merge[df_merge$Decision_1 != df_merge$Decision_2,]
head(diffs_df)

KEY,Proba_no_1,Proba_yes_1,Score_no_1,Score_yes_1,LogProba_no_1,LogProba_yes_1,Decision_1,DecisionProba_1,Score_no_2,Score_yes_2,Proba_no_2,Proba_yes_2,LogProba_no_2,LogProba_yes_2,Decision_2,DecisionProba_2


In [18]:
stopifnot(nrow(diffs_df) == 0)

In [19]:
summary(df_sql_out)

      KEY         Score_no       Score_yes         Proba_no     
 Min.   :  1.00   Mode:logical   Mode:logical   Min.   :0.0000  
 1st Qu.: 38.25   NA's:150       NA's:150       1st Qu.:0.0000  
 Median : 75.50                                 Median :1.0000  
 Mean   : 75.50                                 Mean   :0.6667  
 3rd Qu.:112.75                                 3rd Qu.:1.0000  
 Max.   :150.00                                 Max.   :1.0000  
   Proba_yes       LogProba_no           LogProba_yes     Decision 
 Min.   :0.0000   Min.   :-1.798e+308   Min.   :-100.00   no :100  
 1st Qu.:0.0000   1st Qu.: -3.100e+01   1st Qu.: -71.54   yes: 50  
 Median :0.0000   Median :  0.000e+00   Median : -43.97            
 Mean   :0.3333   Mean   :-3.116e+307   Mean   : -40.49            
 3rd Qu.:1.0000   3rd Qu.:  0.000e+00   3rd Qu.:   0.00            
 Max.   :1.0000   Max.   :  0.000e+00   Max.   :   0.00            
 DecisionProba
 Min.   :1    
 1st Qu.:1    
 Median :1    
 Mean   :

In [20]:
summary(df_r_out)

    Proba_no        Proba_yes           KEY         Score_no      
 Min.   :0.0000   Min.   :0.0000   Min.   :  1.00   Mode:logical  
 1st Qu.:0.0000   1st Qu.:0.0000   1st Qu.: 38.25   NA's:150      
 Median :1.0000   Median :0.0000   Median : 75.50                 
 Mean   :0.6667   Mean   :0.3333   Mean   : 75.50                 
 3rd Qu.:1.0000   3rd Qu.:1.0000   3rd Qu.:112.75                 
 Max.   :1.0000   Max.   :1.0000   Max.   :150.00                 
 Score_yes       LogProba_no      LogProba_yes    Decision  DecisionProba
 Mode:logical   Min.   :-36.04   Min.   :-36.04   no :100   Min.   :1    
 NA's:150       1st Qu.:-36.04   1st Qu.:-36.04   yes: 50   1st Qu.:1    
                Median :  0.00   Median :-36.04             Median :1    
                Mean   :-11.48   Mean   :-23.86             Mean   :1    
                3rd Qu.:  0.00   3rd Qu.:  0.00             3rd Qu.:1    
                Max.   :  0.00   Max.   :  0.00             Max.   :1    